<a href="https://colab.research.google.com/github/yasiruKodikara/machine-learning-lessons/blob/main/Sentiment_Analysis_on_Movie_Reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import nltk


In [ ]:
df = pd.read_csv('IMDB Dataset.csv', encoding='latin-1', engine='python', on_bad_lines='skip')

In [ ]:
df.head(5)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
df.shape

(11038, 2)

In [ ]:
df.rename(columns={'sentiment':'class'},inplace=True)
df.sample(5)

,review,class
3006,"Simply well written, directed and acted... Woo...",positive
9732,This is a complex film that explores the effec...,positive
6887,"I, like many people, saw this film in the thea...",negative
9374,<br /><br />This film was produced and release...,negative
3694,EXTREMITIES <br /><br />Aspect ratio: 1.85:1<b...,negative


In [ ]:
df = df.drop_duplicates(keep='first')
df.groupby('class').describe()

review                                                               
          count unique                                                top freq
class                                                                         
negative   5513   5513  This is meant to be a comedy but mainly bad ta...    1
positive   5505   5505  Typically, "kids" films have some annoying qua...    1

In [ ]:
df.shape

(11018, 2)

In [ ]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
ps = PorterStemmer()

# PreProcessing

In [ ]:
import string

def clean_text(text):
  text = text.lower()
  text = nltk.word_tokenize(text)

  y = []
  for i in text:
    if i.isalnum():
      y.append(i)

  text = y[:]
  y.clear()

  for i in text:
    if i not in stopwords.words('english') and i not in string.punctuation:
      y.append(i)

  text = y[:]
  y.clear()

  for i in text:
    y.append(ps.stem(i))

  return " ".join(y)

In [ ]:
df["cleaned_review"] = df["review"].apply(clean_text)
df.head(10)

,review,class,cleaned_review
0,One of the other reviewers has mentioned that ...,positive,one review mention watch 1 oz episod hook righ...
1,A wonderful little production. <br /><br />The...,positive,wonder littl product br br film techniqu fashi...
2,I thought this was a wonderful way to spend ti...,positive,thought wonder way spend time hot summer weeke...
3,Basically there's a family where a little boy ...,negative,basic famili littl boy jake think zombi closet...
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,petter mattei love time money visual stun film...
5,"Probably my all-time favorite movie, a story o...",positive,probabl favorit movi stori selfless sacrific d...
6,I sure would like to see a resurrection of a u...,positive,sure would like see resurrect date seahunt ser...
7,"This show was an amazing, fresh & innovative i...",negative,show amaz fresh innov idea 70 first air first ...
8,Encouraged by the positive comments about this...,negative,encourag posit comment film look forward watch...
9,If you like original gut wrenching laughter yo...,positive,like origin gut wrench laughter like movi youn...


# Feature Extraction

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf_vec = TfidfVectorizer(max_features=3000)

X = tf_vec.fit_transform(df["cleaned_review"]).toarray()

In [ ]:
X.shape

(11018, 3000)

In [ ]:
y = df["class"].values

# Build the Model

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=2)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(X_train,y_train)

MultinomialNB()

# Lets graph

In [ ]:
# import matplotlib.pyplot as plt

# x = plt.

In [ ]:
from sklearn.metrics import accuracy_score
y_pred = model.predict(X_test)
print(accuracy_score(y_test,y_pred))

0.8466424682395645


In [ ]:
import pickle

# Save the trained model to a file
filename = 'movie_review_sentiment_classfier_model.pkl'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
# Get input text from the user
text = input("Enter the review text: ")

# Preprocess the text using the clean_text function
cleaned_text = clean_text(text)

# Transform the cleaned text using the fitted TF-IDF vectorizer
text_vector = tf_vec.transform([cleaned_text]).toarray()

# Predict the sentiment using the trained model
prediction = model.predict(text_vector)

# Print the prediction
print(f"The predicted sentiment is: {prediction[0]}")

Enter the review text: The Secret Garden is a beautiful, heartwarming reimagining of the classic novel that feels like a breath of fresh air. With stunning visuals, emotionally rich storytelling, and a touch of magical realism, this film is a gentle reminder of the healing power of nature, imagination, and human connection.  The performances are outstanding—particularly from young Dixie Egerickx as Mary Lennox, who brings depth and emotion to her role. The cinematography is absolutely breathtaking, with the garden scenes bursting with life and color, almost becoming a character of its own.  This movie offers a much-needed dose of hope and wonder. Whether you’re familiar with the original story or watching for the first time, The Secret Garden leaves you feeling inspired, uplifted, and deeply moved. Perfect for family viewing or a peaceful evening escape. 
The predicted sentiment is: positive
